In [ ]:
import os

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np
from scipy import spatial

from src.coordinates import sph_to_cart
from src.mappings import encode_normals_rgb
from src.normals import estim_normals_pca
from src.orientation import orient_normals_cvx
from src.plotting import set_defense_context, set_axes_equal, draw_unit_cube

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
# constants

elev, azim = 30, 60  # 3-D view
r = 1  # fixed radius

# Planar model

Figure 2. Skin models for dosimetry analysis: (a) skin models with  different tissue compositions; (b) exposure condition.

K. Li et al., "Intercomparison of Calculated Incident Power Density and Temperature Rise for Exposure From Different Antennas at 10–90 GHz," in IEEE Access, vol. 9, pp. 151654-151666, 2021, doi: 10.1109/ACCESS.2021.3126738.

# Non-planar canonical tissue models

Figure 2. Definitions of the integration volumes for the different absorbed power density calculation schemes. (a) Scheme 1; (b) scheme 2; (c) scheme 3 and (d) scheme 4.

Y. Diao et al., "Assessment of absorbed power density and temperature rise for nonplanar body model under electromagnetic exposure above 6 GHz," in Physics in Medicine & Biology, vol. 65, pp. 224001, 2020, doi: 10.1088/1361-6560/abbdb7.

**Note**.
For practical implementation of the spatially averaged APD in voxel models used in the study, the volumetric integral definition of the APD was adopted.
Four different calculation schemes for the spatially averaged APD for non-planar models are employed.
The bounds of the integration volumes are represented by red polygons.
In subfigures a & b, the upper bound, (L1) is parallel to the grid axis.
In subfigures c & d, L1 is bent along the exposed surface.
In subfigures a & c, side bounds (L2, L3) are parallel to the grid axis, whereas in subfigures b & d, L2
and L3 are parallel to the internal electric field gradients at the model surface.
The lower bound of the integration volume (L4), is defined as the contour where the electric field strength is 1/1000 of the
maximum value in the integration volume.

## Spherical model

In [ ]:
# generate linearly spaced point cloud in the spherical coordinate system

Theta, Phi = np.mgrid[0:2*np.pi:33j,
                      0:np.pi:11j]
x, y, z = sph_to_cart(r, Theta.ravel(), Phi.ravel())
xyz = np.c_[x, y, z]

In [ ]:
# create a convex hull and extract the surface area of a sphere

hull = spatial.ConvexHull(xyz)
print(f'A = {hull.area:.6f}')

In [ ]:
# estimate normals and orient them to point away from the surface

n = estim_normals_pca(xyz, k=10)
n = orient_normals_cvx(xyz, n)
c = encode_normals_rgb(n)

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(3, 3), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax.scatter(*xyz.T, c='k', s=0.5)
    ax.quiver(*xyz.T, *n.T, color='k', length=0.35, lw=0.5)
    hull_triangle_coords = hull.points[hull.simplices]
    triangles = Poly3DCollection(hull_triangle_coords,
                                 ec='gray', fc='gray', lw=0.25, alpha=0.25)
    ax.add_collection3d(triangles)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'sphere-quiver.png'
    fig.savefig(os.path.join('figures', 'model', fname),
                dpi=300, bbox_inches='tight')

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(3, 3), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, c=c, ec='k', lw=0.5, s=20)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'sphere-rgb.png'
    fig.savefig(os.path.join('figures', 'model', fname),
                dpi=300, bbox_inches='tight')

# Cylindrical model

In [ ]:
# generate linearly spaced point cloud in polar coordinates

n = 33
assert n % 3 == 0, 'n must be divisible by 3'
t = np.linspace(0, 2 * np.pi, n, endpoint=False)
x = r * np.cos(t)
y = r * np.sin(t)
xy = np.c_[x, y]
xyz = np.c_[np.repeat(xy, int(n/3), axis=0),
      np.tile(np.linspace(-0.85, 0.85, int(n/3)), n)]

In [ ]:
# create smooth spherical surface

hull = spatial.ConvexHull(xyz)
A_hull = hull.area
print(f'A = {A_hull:.6f}')

In [ ]:
# estimate normals

n = estim_normals_pca(xyz, k=10)
n = orient_normals_cvx(xyz, n)
c = encode_normals_rgb(n)

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(3, 3), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax.scatter(*xyz.T, c='k', s=0.5)
    ax.quiver(*xyz.T, *n.T, color='k', length=0.35, lw=0.5)
    hull_triangle_coords = hull.points[hull.simplices]
    triangles = Poly3DCollection(hull_triangle_coords,
                                 ec='gray', fc='gray', lw=0.25, alpha=0.25)
    ax.add_collection3d(triangles)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'cylinder-quiver.png'
    fig.savefig(os.path.join('figures', 'model', fname),
                dpi=300, bbox_inches='tight')

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(3, 3), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, c=c, ec='k', lw=0.5, s=25)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'cylinder-rgb.png'
    fig.savefig(os.path.join('figures', 'model', fname),
                dpi=300, bbox_inches='tight')

# RGB cube encoding normals

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(2, 2), constrained_layout=False)
    ax = plt.axes(projection ='3d')
    ax = draw_unit_cube(ax)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax.set(xlim=[-0.15, 1.15],
           ylim=[-0.15, 1.15],
           zlim=[-0.15, 1.15])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'rgb-cube.png'
    fig.savefig(os.path.join('figures', 'model', fname),
                dpi=300, bbox_inches='tight')